In [1]:
import pandas as pd
import plotly.express as px
import requests
import json
import streamlit as st
import mysql.connector as sql

# Fetch GeoJSON data
url = "https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson"
response = requests.get(url)
data1 = json.loads(response.content)

# Extract state names and sort them in alphabetical order
state_names = [feature['properties']['ST_NM'] for feature in data1['features']]
state_names.sort()

In [2]:
state_names

['Andaman & Nicobar',
 'Andhra Pradesh',
 'Arunachal Pradesh',
 'Assam',
 'Bihar',
 'Chandigarh',
 'Chhattisgarh',
 'Dadra and Nagar Haveli and Daman and Diu',
 'Delhi',
 'Goa',
 'Gujarat',
 'Haryana',
 'Himachal Pradesh',
 'Jammu & Kashmir',
 'Jharkhand',
 'Karnataka',
 'Kerala',
 'Ladakh',
 'Lakshadweep',
 'Madhya Pradesh',
 'Maharashtra',
 'Manipur',
 'Meghalaya',
 'Mizoram',
 'Nagaland',
 'Odisha',
 'Puducherry',
 'Punjab',
 'Rajasthan',
 'Sikkim',
 'Tamil Nadu',
 'Telangana',
 'Tripura',
 'Uttar Pradesh',
 'Uttarakhand',
 'West Bengal']

In [3]:
# Creating connection with MySQL workbench and retrieve the data from the "agg_trans" table as dataframe:
mydb = sql.connect(host="localhost",
                   user="root", 
                   password="Dhinesh!1", 
                   database="phonepe_pulse")
mycursor = mydb.cursor()

mycursor.execute("SELECT state, year, quarter, transaction_type, transaction_count, transaction_amount FROM agg_trans")
data = mycursor.fetchall()
columns = ["state", "year", "quarter", "transaction_type", "transaction_count", "transaction_amount"]
df = pd.DataFrame(data, columns=columns)
print(df)

#Find unique values in column 'A'
pd.DataFrame(df['state'].unique())

                           state  year  quarter          transaction_type  \
0      andaman-&-nicobar-islands  2018        1  Recharge & bill payments   
1      andaman-&-nicobar-islands  2018        1     Peer-to-peer payments   
2      andaman-&-nicobar-islands  2018        1         Merchant payments   
3      andaman-&-nicobar-islands  2018        1        Financial Services   
4      andaman-&-nicobar-islands  2018        1                    Others   
...                          ...   ...      ...                       ...   
19765                west-bengal  2023        2         Merchant payments   
19766                west-bengal  2023        2     Peer-to-peer payments   
19767                west-bengal  2023        2  Recharge & bill payments   
19768                west-bengal  2023        2        Financial Services   
19769                west-bengal  2023        2                    Others   

       transaction_count  transaction_amount  
0                   4200    

,0
0,andaman-&-nicobar-islands
1,andhra-pradesh
2,arunachal-pradesh
3,assam
4,bihar
5,chandigarh
6,chhattisgarh
7,dadra-&-nagar-haveli-&-daman-&-diu
8,delhi
9,goa


In [4]:
# Assuming your DataFrame 'df' looks like this
# df = pd.DataFrame({'states': [...state_names...]})

# Define the conversion function
def convert_state_name(state):
    return state.replace('-', ' ').title()

# Apply the conversion function to the 'states' column in the DataFrame
df['state'] = df['state'].apply(lambda x: convert_state_name(x))

# Assuming you have already converted and updated the DataFrame 'df'
df.to_csv('D:/Project/2PHONEPE/india_map.csv', index=False)

In [5]:
# Read the "india_map" CSV file into a DataFrame
df1 = pd.read_csv('D:/Project/2PHONEPE/india_map.csv')
df1.head(4)
df1

,state,year,quarter,transaction_type,transaction_count,transaction_amount
0,Andaman & Nicobar Islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,Andaman & Nicobar Islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,Andaman & Nicobar Islands,2018,1,Merchant payments,298,4.525072e+05
3,Andaman & Nicobar Islands,2018,1,Financial Services,33,1.060142e+04
4,Andaman & Nicobar Islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
19765,West Bengal,2023,2,Merchant payments,245111000,1.767046e+11
19766,West Bengal,2023,2,Peer-to-peer payments,240347041,7.970548e+11
19767,West Bengal,2023,2,Recharge & bill payments,58950434,3.478924e+10
19768,West Bengal,2023,2,Financial Services,327537,3.174670e+08


In [7]:

# Assuming df is your DataFrame
min_transaction_amount = df['transaction_amount'].min()
max_transaction_amount = df['transaction_amount'].max()

print("Minimum Transaction Amount:", min_transaction_amount)
print("Maximum Transaction Amount:", max_transaction_amount)

Minimum Transaction Amount: 34.39721181625656
Maximum Transaction Amount: 2393380395296.8516


In [8]:
# Geo plot


fig_tra = px.choropleth(
    df1,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='state',
    color='transaction_amount',
    color_continuous_scale='thermal',
    range_color=(df1['transaction_amount'].min(), df1['transaction_amount'].max()),  # Set custom range
    title='Transaction Analysis'
)

fig_tra.update_geos(fitbounds="locations", visible=False)
fig_tra.update_layout(title_font=dict(size=33), title_font_color='#6739b7', height=800)

st.plotly_chart(fig_tra, use_container_width=True)

2023-09-08 21:42:54.582 
  command:

    streamlit run C:\Users\DK\anaconda3\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [9]:
# Read the "india_map" CSV file into a DataFrame
# Assuming df1 is your DataFrame containing the data
df1 = pd.read_csv(r'D:/Project/2PHONEPE/india_map.csv')
df1.head(4)

# Debugging statements
print("Data Types:")
print(df1.dtypes)

print("Null Values:")
print(df1.isnull().sum())

print("Transaction Amount Summary:")
print(df1['transaction_amount'].describe())

# Calculate the range of transaction_amount for setting zmin and zmax
zmin = df1['transaction_amount'].min()
zmax = df1['transaction_amount'].max()

fig_tra = px.choropleth(
    df1,
    geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
    featureidkey='properties.ST_NM',
    locations='state',
    color='transaction_amount',
    color_continuous_scale='Viridis',  # Change the color scale
    range_color=(zmin, zmax),  # Set custom range
    title='Transaction Analysis'
)

fig_tra.update_geos(fitbounds="locations", visible=False)
fig_tra.update_layout(title_font=dict(size=33), title_font_color='#6739b7', height=800)

st.plotly_chart(fig_tra, use_container_width=True)


Data Types:
state                  object
year                    int64
quarter                 int64
transaction_type       object
transaction_count       int64
transaction_amount    float64
dtype: object
Null Values:
state                 0
year                  0
quarter               0
transaction_type      0
transaction_count     0
transaction_amount    0
dtype: int64
Transaction Amount Summary:
count    1.977000e+04
mean     4.168902e+10
std      1.721274e+11
min      3.439721e+01
25%      3.063466e+07
50%      3.080042e+08
75%      6.799375e+09
max      2.393380e+12
Name: transaction_amount, dtype: float64


DeltaGenerator()

In [23]:
# data query for map_trans :
# Creating connection with MySQL workbench and retrieve the data from the "agg_trans" table as dataframe:
mydb = sql.connect(host="localhost",
                   user="root", 
                   password="Dhinesh!1", 
                   database="phonepe_pulse")
mycursor = mydb.cursor()

mycursor.execute("SELECT state, year, quarter, District, Count, Amount FROM map_trans")
data = mycursor.fetchall()
columns = ["state", "year", "quarter", "District", "Count", "Amount"]
df3 = pd.DataFrame(data, columns=columns)
df3['state'] = df3['state'].apply(lambda x: convert_state_name(x))
df3



,state,year,quarter,District,Count,Amount
0,Andaman & Nicobar Islands,2018,1,north and middle andaman district,442,9.316631e+05
1,Andaman & Nicobar Islands,2018,1,south andaman district,5688,1.256025e+07
2,Andaman & Nicobar Islands,2018,1,nicobars district,528,1.139849e+06
3,Andaman & Nicobar Islands,2018,2,north and middle andaman district,825,1.317863e+06
4,Andaman & Nicobar Islands,2018,2,south andaman district,9395,2.394824e+07
...,...,...,...,...,...,...
80495,West Bengal,2023,2,nadia district,26194658,5.410386e+10
80496,West Bengal,2023,2,birbhum district,14912523,2.941312e+10
80497,West Bengal,2023,2,purba medinipur district,30724502,6.595899e+10
80498,West Bengal,2023,2,maldah district,24677357,5.218621e+10


In [24]:
# download to local drive as csv file ;
df3.to_csv('D:/Project/2PHONEPE/histogram_district.csv', index=False)

In [12]:
# for voline plot :
mycursor.execute("SELECT State, Year, Quarter, District, RegisteredUser, AppOpens FROM map_user")
data = mycursor.fetchall()
columns = ["State", "Year", "Quarter", "District", "RegisteredUser", "AppOpens"]
df4 = pd.DataFrame(data, columns=columns)
df4

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman district,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman district,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars district,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman district,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
80515,west-bengal,2023,2,nadia district,1482400,9509833
80516,west-bengal,2023,2,birbhum district,934943,12789154
80517,west-bengal,2023,2,purba medinipur district,1468608,12096564
80518,west-bengal,2023,2,maldah district,1046132,17662446


In [13]:
# download to local drive as csv file ;
df4.to_csv('D:/Project/2PHONEPE/violin_plot.csv', index=False)

In [25]:
# Assuming your DataFrame 'df' looks like this
# df = pd.DataFrame({'states': [...state_names...]})

# Define the conversion function
def convert_state_name(state):
    return state.replace('-', ' ').title()

# Apply the conversion function to the 'states' column in the DataFrame
df4['State'] = df4['State'].apply(lambda x: convert_state_name(x))

# Assuming you have already converted and updated the DataFrame 'df'
df4.to_csv('D:/Project/2PHONEPE/line_plot.csv', index=False)

In [26]:
# read the "state_map" CSV file into a DataFrame:
df4 = pd.read_csv('D:/Project/2PHONEPE/line_plot.csv')
df4

,State,Year,Quarter,District,RegisteredUser,AppOpens
0,Andaman & Nicobar Islands,2018,1,north and middle andaman district,632,0
1,Andaman & Nicobar Islands,2018,1,south andaman district,5846,0
2,Andaman & Nicobar Islands,2018,1,nicobars district,262,0
3,Andaman & Nicobar Islands,2018,2,north and middle andaman district,911,0
4,Andaman & Nicobar Islands,2018,2,south andaman district,8143,0
...,...,...,...,...,...,...
80515,West Bengal,2023,2,nadia district,1482400,9509833
80516,West Bengal,2023,2,birbhum district,934943,12789154
80517,West Bengal,2023,2,purba medinipur district,1468608,12096564
80518,West Bengal,2023,2,maldah district,1046132,17662446


In [32]:
mycursor = mydb.cursor()

mycursor.execute("SELECT * FROM agg_user")
data = mycursor.fetchall()
columns = ["state", "year", "quarter", "Brands", "Count", "Percentage"]
df5 = pd.DataFrame(data, columns=columns)


In [34]:
df5['state'] = df5['state'].apply(lambda x: convert_state_name(x))
df5

,state,year,quarter,Brands,Count,Percentage
0,Andaman & Nicobar Islands,2018,1,Xiaomi,1665,0.247033
1,Andaman & Nicobar Islands,2018,1,Samsung,1445,0.214392
2,Andaman & Nicobar Islands,2018,1,Vivo,982,0.145697
3,Andaman & Nicobar Islands,2018,1,Oppo,501,0.074332
4,Andaman & Nicobar Islands,2018,1,OnePlus,332,0.049258
...,...,...,...,...,...,...
33655,West Bengal,2022,1,Lenovo,330017,0.015056
33656,West Bengal,2022,1,Infinix,284678,0.012987
33657,West Bengal,2022,1,Asus,280347,0.012790
33658,West Bengal,2022,1,Apple,277752,0.012671


In [38]:
df5.to_csv('D:/Project/2PHONEPE/bar_chart.csv', index=False)
